<a href="https://colab.research.google.com/github/mgnarag/binarization-lowres/blob/main/Testing_one_image.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive")

Mounted at /content/drive


In [ ]:
!ls drive/My\ Drive
file_path = "/content/drive/My Drive/Architectural_designs/From Arki/"

In [3]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import tensorflow as tf


from PIL import Image, ImageOps
Image.MAX_IMAGE_PIXELS = None
import os
import numpy as np
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import pandas as pd

from tensorflow import keras

import time
import os
import cv2
from tensorflow import keras

In [4]:
def crop(im):
    width, height = im.size
    data = []
    for j in range(0,int(height/n_size)):
        for i in range(0,int(width/n_size)):
            im1 = im.crop((0 + (n_size*i), 0 + (n_size*j), n_size + (n_size*i), n_size + (n_size*j)))
            im1 = np.array(im1)
            im1 = im1.astype(np.float32)
            im1 = im1/255
            data.append(im1)
    return data


In [ ]:
gt = Image.open(file_path+ '4 Testing/ground_truth_100dpi.png').convert('L')
img_gt = cv2.imread(file_path+ '4 Testing/ground_truth_100dpi.png')
gt = np.asarray(gt) > 128
gt = ~gt

filename = "nbs_100dpi_wb.png"
directory = file_path + '4 Testing/' + filename
#directory = 'test_output_v'+str(model)+'.png'
image = Image.open(directory)
rgb = image.convert("RGB")
r, g, b = rgb.split()
w_dirty, h_dirty = r.size

gray = ImageOps.grayscale(Image.open(directory))
gray = np.array(gray)
gray = Image.fromarray(gray.astype('uint8'))


dirty = [gray,r,g,b]


METRIC = []
channel = ['gray','red','green', 'blue']
for model in range(1,7):
  for color in range(0,4):
      print('Model = ', model, '', str(channel[color]))
      from tensorflow import keras
      autoencoder = keras.models.load_model(file_path + '1 Models/100 DPI white balanced and bw output/autoencoder_'+str(model)+ '_'+str(channel[color]))
      if model == 1:
          n_size = 32
      if model == 2:
          n_size = 32
      if model == 3:
          n_size = 64
      if model == 4:
          n_size = 64
      if model == 5:
          n_size = 128
      if model == 6:
          n_size = 128
      if model == 7:
          n_size = 256
      if model == 8:
          n_size = 256

      xx = int(w_dirty/n_size)
      final=[]

      for portion in range(0,xx):
          test = dirty[color]
          im1 = test.crop((n_size*portion, 0, (n_size*portion) + n_size, h_dirty))
          w1, h1 = im1.size
          w = int(w1/n_size)
          h = int(h1/n_size)

          neverbeforeseen = np.array(crop(im1))
          encoded_imgs = autoencoder.encoder(neverbeforeseen).numpy()
          decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

          col = np.vstack((decoded_imgs[0],decoded_imgs[1]))
          for i in range(2,h):
              col = np.vstack((col,decoded_imgs[i]))

          y = np.where(col > 0.5,1,0) #round the values
          y = (y * 255).astype('uint8')
          if portion == 0:
              final = y
          if portion > 0:
              final = np.hstack((final,y))

      final = np.squeeze(final)
      reconstructed = Image.fromarray(final)
      #reconstructed.save(file_path + "5 Tested/output v" + str(model) + '_'+str(channel[color]) + '.png')

      output = Image.open(file_path + "5 Tested/output v" + str(model)  + '_'+str(channel[color]) +  '.png').convert('L')  # convert to grayscale
      img_out = cv2.imread(file_path + "5 Tested/output v" + str(model)  + '_'+str(channel[color]) +  '.png')

      output = np.asarray(output) > 128 # Convert to binary images
      output = ~output

      tp = np.sum(output & gt)
      fp = np.sum(output & ~gt)
      fn = np.sum(~output & gt)
      precision = tp / (tp + fp)
      recall = tp / (tp + fn)
      f1score = (2*precision*recall)/(precision + recall)

      I_1 = output & gt
      U_1 = output | gt
      I = np.sum(output & gt)
      U = np.sum(output | gt)
      i_o_u = I/U

      psnr = cv2.PSNR(img_out, img_gt)

      METRIC.append([model, str(channel[color]), precision,recall,f1score,i_o_u,psnr])
  #COMBINING RGB ------------------------------------------------
  print('Model = ', model, '', str('RGB'))
  output_red = (np.asarray(Image.open(file_path + "5 Tested/output v" + str(model)  + '_'+str(channel[0]) +  '.png').convert('L'))>128)
  output_green = (np.asarray(Image.open(file_path + "5 Tested/output v" + str(model)  + '_'+str(channel[1]) +  '.png').convert('L'))>128)
  output_blue = (np.asarray(Image.open(file_path + "5 Tested/output v" + str(model)  + '_'+str(channel[2]) +  '.png').convert('L'))>128)
  output_rgb = ~output_red | ~output_green | ~output_blue

  reconstructed = Image.fromarray(~output_rgb)
  #reconstructed.save(file_path + "5 Tested/output v" + str(model) + '_RGB.png')

  output = Image.open(file_path + "5 Tested/output v" + str(model)  + '_RGB.png').convert('L')  # convert to grayscale
  img_out = cv2.imread(file_path + "5 Tested/output v" + str(model) + '_RGB.png')

  output = np.asarray(output) > 128
  output = ~output

  tp = np.sum(output & gt)
  fp = np.sum(output & ~gt)
  fn = np.sum(~output & gt)
  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f1score = (2*precision*recall)/(precision + recall)

  I_1 = output & gt
  O_1 = output | gt
  I = np.sum(output & gt)
  O = np.sum(output | gt)
  i_o_u = I/O

  psnr = cv2.PSNR(img_out, img_gt)

  METRIC.append([model, str('RGB'), precision,recall,f1score,i_o_u,psnr])
  #COMBINING RGB plus grayscale------------------------------------------------
  print('Model = ', model, '', str('RGB+gray'))
  output_gray = (np.asarray(Image.open(file_path + "5 Tested/output v" + str(model) +  '_gray.png').convert('L'))>128)
  output = output_rgb | ~output_gray

  reconstructed = Image.fromarray(~output)
  #reconstructed.save(file_path + "5 Tested/output v" + str(model) + '_RGB_GRAY.png')

  output = Image.open(file_path + "5 Tested/output v" + str(model)  + '_RGB_GRAY.png').convert('L')  # convert to grayscale
  img_out = cv2.imread(file_path + "5 Tested/output v" + str(model) + '_RGB_GRAY.png')

  output = np.asarray(output) > 128
  output = ~output

  tp = np.sum(output & gt)
  fp = np.sum(output & ~gt)
  fn = np.sum(~output & gt)
  precision = tp / (tp + fp)
  recall = tp / (tp + fn)
  f1score = (2*precision*recall)/(precision + recall)

  I_1 = output & gt
  O_1 = output | gt
  I = np.sum(output & gt)
  O = np.sum(output | gt)
  i_o_u = I/O

  psnr = cv2.PSNR(img_out, img_gt)

  METRIC.append([model, str('RGB+gray'),precision,recall,f1score,i_o_u,psnr])


METRIC = pd.DataFrame(METRIC)

headers = ['Model','Channel','Precision', 'Recall', 'F1 Score', 'IOU','PSNR' ]
METRIC.columns = headers

METRIC.to_csv(file_path + '[B] 100 DPI white balanced and bw output.csv')
print(METRIC)

In [10]:
import time

directory = file_path + "6 Traditional Binarization/nbs_whole_WB.png"
image = Image.open(directory)
rgb = image.convert("RGB")
r, g, b = rgb.split()
w_dirty, h_dirty = r.size


dirty = [r,g,b]

channel = ['gray','red','green', 'blue']
model = 3

start = time.time()

for color in range(0,3):
    autoencoder = keras.models.load_model(file_path + '1 Models/100 DPI white balanced and bw output/autoencoder_'+str(model)+ '_'+str(channel[color]))
    n_size = 32

    xx = int(w_dirty/n_size)
    final=[]

    for portion in range(0,xx):
        test = dirty[color]
        im1 = test.crop((n_size*portion, 0, (n_size*portion) + n_size, h_dirty))
        w1, h1 = im1.size
        w = int(w1/n_size)
        h = int(h1/n_size)

        neverbeforeseen = np.array(crop(im1))
        encoded_imgs = autoencoder.encoder(neverbeforeseen).numpy()
        decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

        col = np.vstack((decoded_imgs[0],decoded_imgs[1]))
        for i in range(2,h):
            col = np.vstack((col,decoded_imgs[i]))

        y = np.where(col > 0.5,1,0) #round the values
        y = (y * 255).astype('uint8')
        if portion == 0:
            final = y
        if portion > 0:
            final = np.hstack((final,y))

    final = np.squeeze(final)
    reconstructed = Image.fromarray(final)
    reconstructed.save(file_path + "6 Traditional Binarization/output v_" +str(channel[color]) + '.png')
#COMBINING RGB ------------------------------------------------
print('Model = ', model, '', str('RGB'))
output_red = (np.asarray(Image.open(file_path + "6 Traditional Binarization/output v_" +str(channel[0]) +  '.png').convert('L'))>128)
output_green = (np.asarray(Image.open(file_path + "6 Traditional Binarization/output v_" +str(channel[1]) +  '.png').convert('L'))>128)
output_blue = (np.asarray(Image.open(file_path + "6 Traditional Binarization/output v_" +str(channel[2]) +  '.png').convert('L'))>128)
output_rgb = ~output_red | ~output_green | ~output_blue

reconstructed = Image.fromarray(~output_rgb)
reconstructed.save(file_path + "6 Traditional Binarization/output_RGB.png")

# record end time
end = time.time()

print("Execution time :",
      (end-start), "seconds")

ValueError: ignored

(4,)


/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:2009: FutureWarning: The input object of type 'Image' is an array-like implementing one of the corresponding protocols (`__array__`, `__array_interface__` or `__array_struct__`); but not a sequence (or 0-D). In the future, this object will be coerced as if it was first converted using `np.array(obj)`. To retain the old behaviour, you have to either modify the type 'Image', or assign to an empty array created with `np.empty(correct_shape, dtype=object)`.
  result = asarray(a).shape
/usr/local/lib/python3.10/dist-packages/numpy/core/fromnumeric.py:2009: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  result = asarray(a).shape


In [14]:
import time



filename = "nbs_whole_WB.png"
directory = file_path + '6 Traditional Binarization/' + filename
#directory = 'test_output_v'+str(model)+'.png'
image = Image.open(directory)
rgb = image.convert("RGB")
r, g, b = rgb.split()
w_dirty, h_dirty = r.size

gray = ImageOps.grayscale(Image.open(directory))
gray = np.array(gray)
gray = Image.fromarray(gray.astype('uint8'))

dirty = [gray,r,g,b]


METRIC = []
channel = ['gray','red','green', 'blue']
for model in range(3,4):
  for color in range(1,4):
      print('Model = ', model, '', str(channel[color]))
      from tensorflow import keras
      autoencoder = keras.models.load_model(file_path + '1 Models/100 DPI white balanced and bw output/autoencoder_'+str(model)+ '_'+str(channel[color]))
      if model == 1:
          n_size = 32
      if model == 2:
          n_size = 32
      if model == 3:
          n_size = 64
      if model == 4:
          n_size = 64
      if model == 5:
          n_size = 128
      if model == 6:
          n_size = 128
      if model == 7:
          n_size = 256
      if model == 8:
          n_size = 256

      xx = int(w_dirty/n_size)
      final=[]

      for portion in range(0,xx):
          test = dirty[color]
          im1 = test.crop((n_size*portion, 0, (n_size*portion) + n_size, h_dirty))
          w1, h1 = im1.size
          w = int(w1/n_size)
          h = int(h1/n_size)

          neverbeforeseen = np.array(crop(im1))
          encoded_imgs = autoencoder.encoder(neverbeforeseen).numpy()
          decoded_imgs = autoencoder.decoder(encoded_imgs).numpy()

          col = np.vstack((decoded_imgs[0],decoded_imgs[1]))
          for i in range(2,h):
              col = np.vstack((col,decoded_imgs[i]))

          y = np.where(col > 0.5,1,0) #round the values
          y = (y * 255).astype('uint8')
          if portion == 0:
              final = y
          if portion > 0:
              final = np.hstack((final,y))

      final = np.squeeze(final)
      reconstructed = Image.fromarray(final)
      reconstructed.save(file_path + "6 Traditional Binarization/output v" + str(model) + '_'+str(channel[color]) + '.png')
  #COMBINING RGB ------------------------------------------------
  print('Model = ', model, '', str('RGB'))
  output_red = (np.asarray(Image.open(file_path + "6 Traditional Binarization/output v" + str(model)  + '_'+str(channel[1]) +  '.png').convert('L'))>128)
  output_green = (np.asarray(Image.open(file_path + "6 Traditional Binarization/output v" + str(model)  + '_'+str(channel[2]) +  '.png').convert('L'))>128)
  output_blue = (np.asarray(Image.open(file_path + "6 Traditional Binarization/output v" + str(model)  + '_'+str(channel[3]) +  '.png').convert('L'))>128)
  output_rgb = ~output_red | ~output_green | ~output_blue

  reconstructed = Image.fromarray(~output_rgb)
  reconstructed.save(file_path + "6 Traditional Binarization/output v" + str(model) + '_RGB.png')

# record end time
end = time.time()

print("Execution time :",
      (end-start), "seconds")


Model =  3  red
Model =  3  green
Model =  3  blue
Model =  3  RGB
Execution time : 1038.7171921730042 seconds
